In [1]:
import torch
torch.cuda.is_available()
torch.__version__

'2.7.0+cu126'

In [10]:
# -*- coding: utf-8 -*-
import torch
from transformers import DataCollatorForLanguageModeling
from datasets import Dataset, load_dataset  # 正确的小写导入
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import pandas as pd

# 基础配置
# model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
model_name = "Qwen/Qwen2.5-1.5B" #参数量和 gpt2 一样.
# output_dir = "./lora_finetuned"
output_dir = "./lora_finetuned_mini"
device_map = "auto"


# # 数据预处理函数
# def format_data(example):
#     system_prompt = example["instruction"]
#     user_input = example["prompt"]
#     response = example["response"]
    
#     full_prompt = f"<系统提示>{system_prompt}</系统提示>\n<输入>{user_input}</输入>\n<响应>{response}</响应>"
#     return {"text": full_prompt}

# 数据预处理函数
def format_data(example):
    system_prompt = example["instruction"]
    user_input = example["prompt"]
    response = example["response"]
    
    full_prompt = f"[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{user_input} [/INST]\n{response}"
    return {"text": full_prompt}

# 加载前10行数据
def load_mini_dataset(csv_path, n_rows=2):
    df = pd.read_csv(csv_path, nrows=n_rows)
    # print(df)
    return Dataset.from_pandas(df)

# 加载迷你数据集
dataset = load_mini_dataset("train_zh.csv")
dataset = dataset.map(format_data, remove_columns=["instruction", "prompt", "response", "meta"])
# # 加载数据集
# dataset = load_dataset("csv", data_files="/home/kchenbw/langrensha/train_zh.csv")["train"]
# dataset = dataset.map(format_data, remove_columns=["instruction", "prompt", "response", "meta"])

# 分割数据集
dataset = dataset.train_test_split(test_size=0.1)

# 加载模型和分词器
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True
)

# 配置LoRA参数
lora_config = LoraConfig(
    r=8, 
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # 根据DeepSeek模型结构调整 
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# 准备模型
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# 数据预处理
def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"  # 确保返回 PyTorch Tensor
    )
    tokenized["labels"] = tokenized["input_ids"].clone()
    
    # 添加类型检查
    # for key in tokenized:
    #     assert isinstance(tokenized[key], torch.Tensor), f"{key} 必须是 Tensor，当前类型: {type(tokenized[key])}"
    
    return tokenized
# 打印第一个样本的结构
# print(tokenized_dataset["train"][0])

# 输出示例（应类似以下结构）
# {'input_ids': tensor([...]), 'attention_mask': tensor([...]), 'labels': tensor([...])}

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"]
)

# 训练参数
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    save_steps=100,
    report_to="tensorboard",
    load_best_model_at_end=True
)
# 使用默认 data_collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # 因为是因果语言模型
)
# 创建Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator
)

# 开始训练
print("开始微调训练...")
trainer.train()

# 保存模型
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"模型已保存至 {output_dir}")

Map: 100%|██████████| 2/2 [00:00<00:00, 338.66 examples/s]

trainable params: 1,089,536 || all params: 1,544,803,840 || trainable%: 0.0705


Map: 100%|██████████| 1/1 [00:00<00:00, 152.93 examples/s]
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


开始微调训练...


c:\Users\ckhqw\miniconda3\envs\pytorch\Lib\site-packages\torch\_dynamo\eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss


模型已保存至 ./lora_finetuned_mini


# 生成任务评估（BLEU/ROUGE/METEOR）

In [14]:
import torch
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score
import jieba  # 中文分词支持

# 加载原始模型和微调模型
base_model = AutoModelForCausalLM.from_pretrained(model_name)
finetuned_model = PeftModel.from_pretrained(base_model, output_dir)

# 加载分词器
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 加载测试数据
df = pd.read_csv("train_zh.csv").tail(2)
test_dataset = Dataset.from_pandas(df)


In [20]:
import logging
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import jieba
from torch.nn import CrossEntropyLoss

# 关闭jieba调试日志
logging.getLogger("jieba").setLevel(logging.WARNING)

# 中文分词工具（改用搜索引擎模式提升召回率）
def chinese_tokenize(text):
    return list(jieba.cut_for_search(text))  # 使用搜索引擎模式[[2]]

# 生成预测文本（增加生成长度）
def generate_response(model, instruction, prompt):
    input_text = f"[INST] <<SYS>>\n{instruction}\n<</SYS>>\n\n{prompt} [/INST]"
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=100)  # 增加生成长度[[1]]
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# 显式设置pad_token_id（避免警告）
tokenizer.pad_token_id = tokenizer.eos_token_id

# 计算BLEU/ROUGE（增加平滑函数和ROUGE-L）
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)
smoother = SmoothingFunction()

def calculate_perplexity(model, tokenizer, prompt, response):
    """
    计算模型对给定prompt和response的困惑度
    """
    full_text = prompt + " " + response  # 拼接输入与响应
    inputs = tokenizer(full_text, return_tensors="pt").to(model.device)
    
    # 构造标签：仅计算response部分的loss
    prompt_len = len(tokenizer(prompt, add_special_tokens=False)['input_ids'])
    labels = inputs['input_ids'].clone()
    labels[:, :prompt_len] = -100  # 忽略prompt部分的loss计算
    
    with torch.no_grad():
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
    
    return torch.exp(loss).item()  # 返回困惑度

for i, example in enumerate(test_dataset):
    instruction = example["instruction"]
    # print(f"指令: {instruction}")
    prompt = example["prompt"]
    # print(f"输入: {prompt}")
    reference = example["response"]
    # print(f"参考答案: {reference}")
    
    # 生成预测
    base_output = generate_response(base_model, instruction, prompt)
    ft_output = generate_response(finetuned_model, instruction, prompt)
    # 新增：计算困惑度
    base_ppl = calculate_perplexity(base_model, tokenizer, prompt, reference)
    ft_ppl = calculate_perplexity(finetuned_model, tokenizer, prompt, reference)
    
    # 分词处理
    ref_tokens = chinese_tokenize(reference)
    base_tokens = chinese_tokenize(base_output)
    ft_tokens = chinese_tokenize(ft_output)
    
    # BLEU优化：使用BLEU-2+平滑函数
    bleu_base = sentence_bleu(
        [ref_tokens], base_tokens, 
        weights=(0.5, 0.5),  # 使用BLEU-2
        smoothing_function=smoother.method1  # 添加平滑
    )
    bleu_ft = sentence_bleu(
        [ref_tokens], ft_tokens,
        weights=(0.5, 0.5),
        smoothing_function=smoother.method1
    )
    
    # ROUGE优化：使用分词后的文本并增加ROUGE-L
    rouge_base = scorer.score(
        " ".join(ref_tokens), 
        " ".join(base_tokens)
    )
    rouge_ft = scorer.score(
        " ".join(ref_tokens), 
        " ".join(ft_tokens)
    )
    # 新增打印内容：对比生成结果与参考答案
    print(f"\n{'='*20} 样本{i+1} 原始内容 {'='*20}")
    # print(f"【指令】: {instruction}")
    # print(f"【输入】: {prompt}")
    print(f"【参考回答】: {reference}")
    print(f"\n【基础模型生成】: {base_output}")
    print(f"【微调模型生成】: {ft_output}")
    
    # 评估指标打印（保持原有格式）
    print(f"\n{'='*20} 样本{i+1} 评估结果 {'='*20}")
    print(f"BLEU-2 - 原始: {bleu_base:.4f}, 微调: {bleu_ft:.4f}")
    print(f"ROUGE-2 F1 - 原始: {rouge_base['rouge2'].fmeasure:.4f}, 微调: {rouge_ft['rouge2'].fmeasure:.4f}")
    print(f"ROUGE-L F1 - 原始: {rouge_base['rougeL'].fmeasure:.4f}, 微调: {rouge_ft['rougeL'].fmeasure:.4f}")
    print("-"*50 + "\n")
    
    # 打印困惑度结果
    print(f"Perplexity - 原始: {base_ppl:.2f}, 微调: {ft_ppl:.2f}")
    print("-"*50 + "\n")
    
    # print(f"样本{i+1}评估结果:")
    # print(f"BLEU-2 - 原始: {bleu_base:.4f}, 微调: {bleu_ft:.4f}")
    # print(f"ROUGE-2 F1 - 原始: {rouge_base['rouge2'].fmeasure:.4f}, 微调: {rouge_ft['rouge2'].fmeasure:.4f}")
    # print(f"ROUGE-L F1 - 原始: {rouge_base['rougeL'].fmeasure:.4f}, 微调: {rouge_ft['rougeL'].fmeasure:.4f}")
    # print("-"*50)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



==================== 样本1 原始内容 ====================
【参考回答】: {"想要展示的身份": "村民", "身份标签": {"1号玩家": "未知身份", "2号玩家": "未知身份", "3号玩家": "未知身份", "4号玩家": "未知身份", "5号玩家": "未知身份", "6号玩家": "村民和狼人", "7号玩家": "猎人", "9号玩家": "未知身份"}, "归票": "无", "发言": "我是村民。目前只知道出局的7号是猎人，但是没有办法定义7号带走的6号是什么身份，我会听一下后置位的发言，再去考虑站边问题。"}

【基础模型生成】: [INST] <<SYS>>

你现在正在玩一种叫做“狼人杀”的游戏。
在这款游戏中，玩家通常被分为两个阵营：狼人和村民。
狼人杀游戏中不同角色的玩家有不同的目标：
- 村民的目的是识别出狼人，并通过投票使他们出局。
- 对于狼人来说，他们的主要目标是隐藏他们的真实身份，在讨论中误导他人，以免被投票出局并尽可能的猎杀村民。
以下是一些基本规则：
- 身份：玩家的身份是秘密分配的。狼人彼此知道对方的身份，而村民只知道自己的身份。
- 昼夜更替：游戏有交替的白天和黑夜阶段。夜里，狼人秘密选择一名村民猎杀。白天，所有玩家讨论并投票决定他们认为是狼人的玩家，票数最多的玩家被淘汰。
- 特殊角色：游戏中有存在一些有特殊能力的角色，比如能得知玩家身份的“预言家”等。
- 获胜条件：当游戏中有一个群体实现它们的获胜条件时游戏结束。如果所有狼人被淘汰，村民就获胜。如果狼人杀死了所有普通村民或所有特殊角色，狼人就获胜。

在这个游戏中，我们有从1到9号共9名玩家 —— 6名村民和3名狼人。村民中有特殊角色，包括：
- 1位预言家：
    - 目标：预言家的目的是帮助村民识别狼人。
    - 能力：在夜晚阶段，预言家可以秘密选择一名玩家，每晚了解他的真实身份（是否为狼人）。
- 1位女巫：
    - 目标：女巫的目的是策略性地使用她的特殊能力来帮助村民。
    - 能力：女巫有一瓶解药和一瓶毒药。一旦使用，后续回合中不能再用。女巫不能在同一晚既使用解药又使用毒药。解药可以用来救一名在夜间被狼人猎杀的玩家。毒药可以淘汰一名很可能是狼人的玩家。
- 1位猎人：
    - 目标

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



==================== 样本2 原始内容 ====================
【参考回答】: {"想要展示的身份": "村民", "身份标签": {"1号玩家": "未知身份", "2号玩家": "未知身份", "3号玩家": "未知身份", "4号玩家": "村民", "5号玩家": "预言家", "6号玩家": "村民和狼人", "7号玩家": "猎人", "8号玩家": "未知身份"}, "归票": "无", "发言": "我9号底牌是村民身份，昨晚7号猎人吃刀，女巫没有使用解药。我不知道被7号猎人带走的6号玩家是什么身份，6号可能是村民也有可能是狼人，希望6号玩家是个狼人。5号玩家第一个发言跳预言家报4号金水，我认为5号还是比较有力度的，不怕4号反水立警。我认为5号是真预言家可能性较大，我听一下后置位玩家的发言来进行投票。"}

【基础模型生成】: [INST] <<SYS>>

你现在正在玩一种叫做“狼人杀”的游戏。
在这款游戏中，玩家通常被分为两个阵营：狼人和村民。
狼人杀游戏中不同角色的玩家有不同的目标：
- 村民的目的是识别出狼人，并通过投票使他们出局。
- 对于狼人来说，他们的主要目标是隐藏他们的真实身份，在讨论中误导他人，以免被投票出局并尽可能的猎杀村民。
以下是一些基本规则：
- 身份：玩家的身份是秘密分配的。狼人彼此知道对方的身份，而村民只知道自己的身份。
- 昼夜更替：游戏有交替的白天和黑夜阶段。夜里，狼人秘密选择一名村民猎杀。白天，所有玩家讨论并投票决定他们认为是狼人的玩家，票数最多的玩家被淘汰。
- 特殊角色：游戏中有存在一些有特殊能力的角色，比如能得知玩家身份的“预言家”等。
- 获胜条件：当游戏中有一个群体实现它们的获胜条件时游戏结束。如果所有狼人被淘汰，村民就获胜。如果狼人杀死了所有普通村民或所有特殊角色，狼人就获胜。

在这个游戏中，我们有从1到9号共9名玩家 —— 6名村民和3名狼人。村民中有特殊角色，包括：
- 1位预言家：
    - 目标：预言家的目的是帮助村民识别狼人。
    - 能力：在夜晚阶段，预言家可以秘密选择一名玩家，每晚了解他的真实身份（是否为狼人）。
- 1位女巫：
    - 目标：女巫的目的是策略性地使用她的特殊能力来帮助村民。
    - 能力：女巫有一瓶解药和一瓶毒药。一